In [ ]:
# Bibliotecas
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

In [ ]:
# Cria um objeto Service (do Selenium), que é necessário para inicializar o driver do navegador
service = Service()

# Cria configurações para o navegador
options = webdriver.ChromeOptions()

# Inicializa o Chrome com o service e as opções definidas
driver=webdriver.Chrome(service=service, options=options)

# Define a URL que será acessada pelo navegador
url='https://www.bcorporation.net/en-us/find-a-b-corp/'

# Acessa a URL com o navegador automatizado
driver.get(url)

In [ ]:
# Busca por elementos com diferentes seletores isso muda de página para página, neste caso eu precisava pegar três informações diferentes
elementos = (
    driver.find_elements(By.TAG_NAME, 'h1') + 
    driver.find_elements(By.CLASS_NAME, 'opacity-60') + 
    driver.find_elements(By.CLASS_NAME, 'mt-8')
)

# Filtra os elementos coletados, extraindo apenas os textos não vazios.
# .text = pega o texto do elemento
# .strip() = remove espaços em branco no começo e no fim
# if el.text.strip() != '' = ignora elementos sem texto
textos = [el.text.strip() for el in elementos if el.text.strip() != '']

# Mostra a lista final com os textos extraídos
textos

In [ ]:
# Lista para armazenar informações coletadas
lista_empresas = []

# Loop para percorrer todas as páginas
while True:
    try:
        # Captura os elementos com o nome da empresa (SEM guardar em variável fixa)
        titleElements = driver.find_elements(By.CSS_SELECTOR, 'span[data-testid="company-name-desktop"]')

        # Percorre cada empresa encontrada na página
        for i in range(len(titleElements)):
            try:
                # Reencontra os elementos novamente (evita erro de referência obsoleta)
                titleElements = driver.find_elements(By.CSS_SELECTOR, 'span[data-testid="company-name-desktop"]')

                # Aguarda o elemento estar clicável e clica nele
                WebDriverWait(driver, 10).until(EC.element_to_be_clickable(titleElements[i])).click()

                time.sleep(1)  # Pausa para garantir que a nova página carregue

                # Coleta informações da página da empresa
                info_elements = (
                    driver.find_elements(By.TAG_NAME, 'h1') +
                    driver.find_elements(By.CLASS_NAME, 'opacity-60') +
                    driver.find_elements(By.CLASS_NAME, 'mt-8')
                )

                # Filtra e guarda apenas os textos não vazios
                lista_empresas.append([el.text.strip() for el in info_elements if el.text.strip()])

                driver.back()  # Volta para a lista de empresas
                time.sleep(1)  # Aguarda o recarregamento da página

            except Exception as e:
                print(f"Erro ao processar a empresa {i}: {e}")

        # Tenta clicar no botão "Next" para ir à próxima página
        try:
            next_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, 'button[aria-label="Next"]'))
            )
            next_button.click()
            time.sleep(1)  # Aguarda carregamento da nova página

        except Exception:
            print("Botão 'Next' não encontrado. Finalizando coleta.")
            break  # Sai do loop se não houver mais páginas

    except Exception as e:
        print(f"Erro inesperado: {e}")
        break

# Finaliza o navegador após concluir a coleta
driver.quit()

In [ ]:
# Mostra a lista final com os textos extraídos
lista_empresas

In [ ]:
# Transforma a lista (lista_empresas) em uma tabela (dataframe)
df_empresas = pd.DataFrame(lista_empresas, columns=[
    'Nome', 'Localização', 'Data do certificado', 'Industria', 'Setor', 
    'Operação', 'Site', 'Descrição'
])

# Exibe o dataframe (Primeiras 5 linhas)
df_empresas.head()

In [ ]:
# Transforma o dataframe em uma planilha em excel .xlsx
df_empresas.to_excel('Empresas_B.xlsx', index=False)